In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb

In [ ]:
data = pd.read_csv('../train.csv')
X = pd.read_csv('X_before_stack.csv')
test = pd.read_csv('test_before_stack.csv')
y = pd.read_csv('y.csv').values[:, 1]

In [ ]:
del X['Unnamed: 0']
del X['Unnamed: 0.1']
del test['Unnamed: 0']
del test['Unnamed: 0.1']

In [ ]:
X['dist'].describe()

In [ ]:
X[X['dist']>0]['dist'].values

In [ ]:
dists_log = np.log(X[X['dist']>0]['dist'].values)

In [ ]:
trsh = plt.hist(dists_log, bins=1000)

In [ ]:
dists_log = np.log( np.array(X['dist'].values) + np.array([2.]*len(X)) )

In [ ]:
trsh=plt.hist(dists_log, bins=1000)

In [ ]:
test.head(1)

In [ ]:
X.head(1)

In [ ]:
X['isdist0'] = np.array([1 if x[0] == 0 else 0 for x in X.values])
X['isdistminus1'] = np.array([1 if x[0] == -1 else 0 for x in X.values])

test['isdist0'] = np.array([1 if x[0] == 0 else 0 for x in test.values])
test['isdistminus1'] = np.array([1 if x[0] == -1 else 0 for x in test.values])

# Cancel_time

In [ ]:
cancel_times = data['cancel_time']
cancel_times = np.log(cancel_times + np.array([2.]*len(cancel_times)))

In [ ]:
# params = {} 
# params['objective']           = "reg:linear"
# params['booster']             = "gbtree"
# params['eta']                 = 0.05
# params['max_depth']           = 7
# params['subsample']           = 0.95
# params['colsample_bytree']    = 0.95
# params['seed']                = 1337
# params['njobs']               = -1
# params['silent']              = 1
params = {} 
params['objective']           = "reg:linear"
# params['booster']             = "gbtree"
# params['eval_metric']         = "auc"
params['eta']                 = 0.1
# params['max_depth']           = 7
# params['subsample']           = 0.681
# params['colsample_bytree']    = 0.95
params['seed']                = 1337
params['njobs']               = -1
# params['silent']              = 1

In [ ]:
            # bound = int(X.shape[0]*0.9)
            # x_train, x_val = X[:bound], X[bound:]
            # y_train, y_val = cancel_times[:bound], cancel_times[bound:]

            # dtrain = xgb.DMatrix(x_train, y_train)
            # dval = xgb.DMatrix(x_val, y_val)
            # watchlist = [(dval,'eval')]

            # num_trees = 3000
            # esr = 10
            # gbm = xgb.train(params, dtrain, num_trees, evals=watchlist, 
            #                 verbose_eval=True, early_stopping_rounds = esr)

In [ ]:
dtrain = xgb.DMatrix(X, cancel_times)
gbm = xgb.train(params, dtrain, 25)

In [ ]:
dtrain = xgb.DMatrix(X)
train_pred_cancel_time = gbm.predict(dtrain, ntree_limit=25)
pd.DataFrame(train_pred_cancel_time).to_csv('feature_train_cancel_time.csv')

In [ ]:
dtest = xgb.DMatrix(test)
pred_cancel_time = gbm.predict(dtest, ntree_limit=25)
pd.DataFrame(pred_cancel_time).to_csv('feature_test_cancel_time.csv')

# Driver_found

In [ ]:
driver_founds = data['driver_found']

In [ ]:
# params = {} 
# params['objective']           = "binary:logistic"
# params['booster']             = "gbtree"
# params['eval_metric']         = "auc"
# params['eta']                 = 0.05
# params['max_depth']           = 7
# params['subsample']           = 0.95
# params['colsample_bytree']    = 0.95
# params['seed']                = 1337
# params['njobs']               = -1
params = {} 
params['objective']           = "binary:logistic"
# params['booster']             = "gbtree"
# params['eval_metric']         = "auc"
params['eta']                 = 0.05
# params['max_depth']           = 7
# params['subsample']           = 0.681
# params['colsample_bytree']    = 0.95
params['seed']                = 1337
params['njobs']               = -1
# params['silent']              = 1

In [ ]:
            # bound = int(X.shape[0]*0.9)
            # x_train, x_val = X[:bound], X[bound:]
            # y_train, y_val = driver_founds[:bound], driver_founds[bound:]

            # dtrain = xgb.DMatrix(x_train, y_train)
            # dval = xgb.DMatrix(x_val, y_val)
            # # dtest = xgb.DMatrix(test)
            # watchlist = [(dval,'eval')]
            # num_trees = 300
            # esr = 10
            # gbm = xgb.train(params, dtrain, num_trees, 
            #                 evals=watchlist, verbose_eval=True, 
            #                 early_stopping_rounds = esr)

In [ ]:
dtrain = xgb.DMatrix(X, driver_founds)
gbm = xgb.train(params, dtrain, 28)

In [ ]:
dtrain = xgb.DMatrix(X)
train_pred_driver_found = gbm.predict(dtrain, ntree_limit=28)
pd.DataFrame(train_pred_driver_found).to_csv('feature_train_driver_found.csv')

In [ ]:
dtest = xgb.DMatrix(test)
pred_driver_found = gbm.predict(dtest, ntree_limit=28)
pd.DataFrame(pred_driver_found).to_csv('feature_test_driver_found.csv')